In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("SF_Crime").getOrCreate()

In [3]:
file_path = "./police-department-calls-for-service.json"
df = spark.read.option("multiLine", True).json(file_path)

In [4]:
df.printSchema()

root
 |-- address: string (nullable = true)
 |-- address_type: string (nullable = true)
 |-- agency_id: string (nullable = true)
 |-- call_date: string (nullable = true)
 |-- call_date_time: string (nullable = true)
 |-- call_time: string (nullable = true)
 |-- city: string (nullable = true)
 |-- common_location: string (nullable = true)
 |-- crime_id: string (nullable = true)
 |-- disposition: string (nullable = true)
 |-- offense_date: string (nullable = true)
 |-- original_crime_type_name: string (nullable = true)
 |-- report_date: string (nullable = true)
 |-- state: string (nullable = true)



In [5]:
df.count()

199999

In [6]:
df.show(1)

+--------------------+------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+
|             address|address_type|agency_id|           call_date|      call_date_time|call_time|         city|common_location| crime_id|disposition|        offense_date|original_crime_type_name|         report_date|state|
+--------------------+------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+
|Geary Bl/divisade...|Intersection|        1|2018-12-31T00:00:...|2018-12-31T23:57:...|    23:57|San Francisco|               |183653763|        ADM|2018-12-31T00:00:...|            Traffic Stop|2018-12-31T00:00:...|   CA|
+--------------------+------------+---------+--------------------+--------------------+---------+-----------

In [7]:
df.select("call_date", "call_time", "disposition", "original_crime_type_name").show()

+--------------------+---------+------------+------------------------+
|           call_date|call_time| disposition|original_crime_type_name|
+--------------------+---------+------------+------------------------+
|2018-12-31T00:00:...|    23:57|         ADM|            Traffic Stop|
|2018-12-31T00:00:...|    23:54|         CIT|     Traf Violation Cite|
|2018-12-31T00:00:...|    23:49|         HAN|            Passing Call|
|2018-12-31T00:00:...|    23:47|         PAS|           Audible Alarm|
|2018-12-31T00:00:...|    23:46|         CIT|            Traffic Stop|
|2018-12-31T00:00:...|    23:38|          ND|            Passing Call|
|2018-12-31T00:00:...|    23:38|         CIT|            Traffic Stop|
|2018-12-31T00:00:...|    23:36|         HAN|            Traffic Stop|
|2018-12-31T00:00:...|    23:34|Not recorded|            Passing Call|
|2018-12-31T00:00:...|    23:33|         GOA|         Fight No Weapon|
|2018-12-31T00:00:...|    23:33|         HAN|            Traffic Stop|
|2018-

In [14]:
df.select("original_crime_type_name").groupBy(df.original_crime_type_name).count().orderBy('count', ascending=False).show()

+------------------------+-----+
|original_crime_type_name|count|
+------------------------+-----+
|            Passing Call|35180|
|            Traffic Stop|13538|
|     Traf Violation Cite|10921|
|       Suspicious Person|10007|
|      Homeless Complaint| 6072|
|              Trespasser| 5840|
|           Audible Alarm| 5775|
|                  22500e| 5651|
|        Well Being Check| 5552|
|         Muni Inspection| 5509|
|      Suspicious Vehicle| 5031|
|         Fight No Weapon| 4199|
|          Noise Nuisance| 4072|
|      Auto Boost / Strip| 2969|
|      Traf Violation Tow| 2915|
|                    Poss| 2851|
|      Mentally Disturbed| 2585|
|             Petty Theft| 2549|
|          Meet W/citizen| 2414|
|       Assault / Battery| 2356|
+------------------------+-----+
only showing top 20 rows



In [17]:
file2 = "./radio_code.json"
df_code = spark.read.json(file2, multiLine=True)

In [18]:
df_code.printSchema()

root
 |-- description: string (nullable = true)
 |-- disposition_code: string (nullable = true)



In [19]:
df_code.show(5)

+-----------+----------------+
|description|disposition_code|
+-----------+----------------+
|     Abated|             ABA|
| Admonished|             ADM|
|    Advised|             ADV|
|     Arrest|             ARR|
|     Cancel|             CAN|
+-----------+----------------+
only showing top 5 rows



In [27]:
df.join(df_code, df.disposition==df_code.disposition_code, how='right').show(5)

+--------------------+---------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+-----------+----------------+
|             address|   address_type|agency_id|           call_date|      call_date_time|call_time|         city|common_location| crime_id|disposition|        offense_date|original_crime_type_name|         report_date|state|description|disposition_code|
+--------------------+---------------+---------+--------------------+--------------------+---------+-------------+---------------+---------+-----------+--------------------+------------------------+--------------------+-----+-----------+----------------+
|500 Block Of Jack...|Premise Address|        1|2018-12-31T00:00:...|2018-12-31T23:21:...|    23:21|San Francisco|               |183653664|        ABA|2018-12-31T00:00:...|       Assault / Battery|2018-12-31T00:00:...|   CA|     Abate